<h1>NBA Game Prediction</h1>
This project's goal is to predict the outcome of NBA games, and it currently does so with 65-66% accuracy. It's an <u><b>ongoing</b></u> project as I'd like to add/change many things to improve the speed, accuracy, etc. 

The following is a list of potential future improvements/ideas/reminders in no particular order:

- Optimize window_size hyperparameter
- Experiment more with other models (logistic regression has been the best so far but SVM and gradient boosting were close even though I barely experimented with their hyperparameters)
- Consider removing outliers?
- Experiment more with feature selection (everything I've tried so far made such a small change that it's hard to tell whether it was coincidental or not)
- Engineer more features
- Improve performance of combine_with_opp function
- Add way to get predictions for upcoming games for the current season and update model with the latest data
- Could training the model by averaging the same stat over multiple different window sizes possibly be useful? (despite multicollinearity)
- Experiment with weighted mean for calculating stats instead of regular mean
- Search for more stats to add to the model, and even consider obscure ones like longest scoring drought, time spent leading the game, etc.
- Include stats on individual players (not just team stats), in particular include stats regarding player injuries
- Add Elo rating stat?
- Find alternative data source to speed up and simplify the data processing and collection

In [1]:
import os
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt

# sklearn tools
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV

# models:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

This ensures all columns are visible and that not too many rows are printed:

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

The following scrapes the NBA's website to get the various team names and their corresponding acronyms. You'll notice that I replaced the acronyms for the Nets, Hornets, and Suns, since Basketball Reference (which I plan to use for most of the data collection) uses different acronyms. The reason I got the acronyms from the NBA's website in the first place was so I could practice scraping different websites.

In [3]:
def get_teams_dict():    
    response = requests.get("https://www.nba.com/teams")
    soup = BeautifulSoup(response.content, "html.parser")
    
    teams = {}
    for fig in soup.find_all(class_="TeamFigure_tf__jA5HW"):
        abbr = fig.find_all(class_="TeamFigureLink_teamFigureLink__uqnNO")[3].get("data-content")
        name = fig.find(class_="TeamFigure_tfMainLink__OPLFu").text
        teams[abbr] = name

    # replacing the NBA's acronyms with those used by Basketball Reference since that's what I've used for the remaining scraping
    teams["BRK"] = teams.pop("BKN")
    teams["CHO"] = teams.pop("CHA")
    teams["PHO"] = teams.pop("PHX")

    teams = dict(sorted(teams.items()))
    
    return teams

In [4]:
teams = get_teams_dict()
print(teams)

{'ATL': 'Atlanta Hawks', 'BOS': 'Boston Celtics', 'BRK': 'Brooklyn Nets', 'CHI': 'Chicago Bulls', 'CHO': 'Charlotte Hornets', 'CLE': 'Cleveland Cavaliers', 'DAL': 'Dallas Mavericks', 'DEN': 'Denver Nuggets', 'DET': 'Detroit Pistons', 'GSW': 'Golden State Warriors', 'HOU': 'Houston Rockets', 'IND': 'Indiana Pacers', 'LAC': 'LA Clippers', 'LAL': 'Los Angeles Lakers', 'MEM': 'Memphis Grizzlies', 'MIA': 'Miami Heat', 'MIL': 'Milwaukee Bucks', 'MIN': 'Minnesota Timberwolves', 'NOP': 'New Orleans Pelicans', 'NYK': 'New York Knicks', 'OKC': 'Oklahoma City Thunder', 'ORL': 'Orlando Magic', 'PHI': 'Philadelphia 76ers', 'PHO': 'Phoenix Suns', 'POR': 'Portland Trail Blazers', 'SAC': 'Sacramento Kings', 'SAS': 'San Antonio Spurs', 'TOR': 'Toronto Raptors', 'UTA': 'Utah Jazz', 'WAS': 'Washington Wizards'}


This next code block is used for scraping tables from Basketball Reference. The first_header_index parameter is used to ignore the first few headers since Basketball Reference sometimes uses double layered headers.

In [5]:
def get_table(url, first_header_index):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table")
    
    # initialize the data with just the column headers (for now)
    data = [[th.text.strip() for th in table.find("thead").findAll("th")][first_header_index:]]

    # iterate over rows and add to data list
    for row in table.find("tbody").findAll("tr"):
        row_data = [td.text.strip() for td in row.findAll("td")]
        if row_data: 
            data.append(row_data)

    df = pd.DataFrame(data[1:], columns=data[0])
    
    return df

Here is the function used to scrape gamelogs (both regular and advanced) from Basketball Reference. Notice that the request interval only allows a page request once every 30 seconds. This ensures that we respect the website's other users as well as the site itself. To avoid ever having to wait for this lengthy process again, we save everything locally as CSV files.

In [6]:
def scrape_gamelogs(teams, start_year, end_year, request_interval=30):    
    for year in range(start_year, end_year + 1):
        year_folder_path = os.path.join("gamelogs", str(year))
        regular_subfolder_path = os.path.join(year_folder_path, "regular")
        advanced_subfolder_path = os.path.join(year_folder_path, "advanced")
        os.makedirs(year_folder_path)
        os.makedirs(regular_subfolder_path)
        os.makedirs(advanced_subfolder_path)

        for team_abbr in teams:
            url = f"https://www.basketball-reference.com/teams/{team_abbr}/{year}/gamelog/"
            regular_gamelogs = get_table(url, 7) # ignores first 7 columns
            path = os.path.join(regular_subfolder_path, f"{team_abbr}.csv")
            regular_gamelogs.to_csv(path, index=False)
            time.sleep(request_interval) 

            url = f"https://www.basketball-reference.com/teams/{team_abbr}/{year}/gamelog-advanced/"
            advanced_gamelogs = get_table(url, 9) # ignores first 9 columns
            path = os.path.join(advanced_subfolder_path, f"{team_abbr}.csv")
            advanced_gamelogs.to_csv(path, index=False)
            time.sleep(request_interval)

Only run this if you're prepared to wait for a long time:

In [81]:
scrape_gamelogs(teams, 2015, 2023)

For each team and each year we now have statistics (both regular and advanced) for every regular-season game in the span of years desired. I'll now go through the process of converting these many tables into a single table which we'll use to predict NBA game winners.

Firstly, I combine each regular table with its corresponding advanced table so that there's only one table for each team for each year:

In [7]:
def combine_and_extract_relevant_data(year):
    gamelogs = {}
    regular_path = os.path.join("gamelogs", str(year), "regular")
    advanced_path = os.path.join("gamelogs", str(year), "advanced")

    # get all regular and advanced log files while ignoring anything else in the directory
    regular_log_files = [x for x in sorted(os.listdir(regular_path)) if x.endswith(".csv")]
    advanced_log_files = [x for x in sorted(os.listdir(advanced_path)) if x.endswith(".csv")]
    
    for regular_file, advanced_file in zip(regular_log_files, advanced_log_files):
        f1_path = os.path.join(regular_path, regular_file)
        f2_path = os.path.join(advanced_path, advanced_file)
    
        # converts both gamelog types to pandas dataframe
        df1 = pd.read_csv(f1_path)
        df2 = pd.read_csv(f2_path)
    
        # ignores opponent stats to avoid duplicatating data 
        relevant_regular_data = df1.loc[:, :"PF"].drop(columns="Opp.1")

        # ignores "defensive four factors" (except for DRB%) to avoid duplicating data 
        relevant_advanced_data = pd.concat([df2.loc[:, "ORtg":"FT/FGA"], df2.loc[:, "DRB%"]], axis=1).dropna(axis=1, how="all")

        # combines the regular and advanced features into one df
        df = pd.concat([relevant_regular_data, relevant_advanced_data], axis=1)
    
        # the key is the team's abbreviation (3 letters)
        gamelogs[regular_file[:3]] = df 
    
    return gamelogs

Below you can see the Los Angeles Lakers 2023 gamelogs, where the regular data (columns FG - PF) is combined with the advanced data (columns ORtg - DRB%):

In [8]:
gamelogs = combine_and_extract_relevant_data(2023)
gamelogs["LAL"]

,G,Date,Unnamed: 2,Opp,W/L,Tm,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,DRB%
0,1,2022-10-18,@,GSW,L,109,40,94,0.426,10,40,0.250,19,25,0.760,9,48,23,12,4,21,18,96.0,108.3,113.6,0.266,0.426,0.519,50.0,57.5,10.6,7.4,0.479,16.7,19.6,0.202,78.0
1,2,2022-10-20,NaN,LAC,L,97,33,94,0.351,9,45,0.200,22,26,0.846,8,38,23,13,5,10,24,93.3,99.1,104.0,0.277,0.479,0.460,39.6,69.7,12.5,9.8,0.399,8.7,14.3,0.234,75.0
2,3,2022-10-23,NaN,POR,L,104,41,92,0.446,6,33,0.182,16,20,0.800,10,40,26,9,9,13,24,103.8,105.8,100.2,0.217,0.359,0.516,43.5,63.4,9.0,15.8,0.478,11.4,20.4,0.174,69.8
3,4,2022-10-26,@,DEN,L,99,40,90,0.444,8,30,0.267,11,16,0.688,7,47,25,8,4,14,16,96.7,107.4,102.4,0.178,0.333,0.510,46.5,62.5,7.8,6.9,0.489,12.6,13.5,0.122,81.6
4,5,2022-10-28,@,MIN,L,102,37,89,0.416,9,29,0.310,19,27,0.704,12,55,20,8,5,22,21,96.7,105.2,105.5,0.303,0.326,0.506,52.9,54.1,7.6,8.5,0.466,17.9,25.5,0.213,75.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,2023-04-02,@,HOU,W,134,52,93,0.559,9,30,0.300,21,28,0.750,11,51,36,4,8,5,9,138.8,112.9,96.5,0.301,0.323,0.636,53.7,69.2,4.1,11.8,0.608,4.5,28.2,0.226,71.4
78,79,2023-04-04,@,UTA,W,135,48,96,0.500,12,34,0.353,27,35,0.771,11,44,27,6,7,11,21,125.0,123.1,97.8,0.365,0.354,0.606,51.8,56.3,5.6,12.3,0.563,9.0,23.9,0.281,84.6
79,80,2023-04-05,@,LAC,L,118,45,89,0.506,12,29,0.414,16,24,0.667,10,44,27,4,3,14,17,117.4,124.3,100.6,0.270,0.326,0.593,53.7,60.0,4.0,6.5,0.573,12.3,23.3,0.180,87.2
80,81,2023-04-07,NaN,PHO,W,121,45,92,0.489,18,36,0.500,13,17,0.765,10,54,25,6,6,15,12,117.4,103.8,103.0,0.185,0.391,0.608,53.5,55.6,5.8,8.8,0.587,13.1,22.7,0.141,77.2


Secondly, we rename some columns, represent binary data numerically (ex. W or L becomes 1 or 0 respectively), adjust game stats based on the game's pace, and add a feature that checks whether or not a team is playing a back to back game:

In [9]:
def preliminary_preprocessing(gamelogs):
    for team_name, df in gamelogs.items():

        df.columns = df.columns.str.lower() # make column titles lowercase

        df.rename(columns={"unnamed: 2":"location"}, inplace=True)
        df.location = df.location.isna().astype(int) # convert location (home vs away) column to 1s and 0s

        df.insert(df.columns.get_loc("location"), "team", team_name) # add team name to table

        # w/l becomes w_or_l to avoid special character
        # tm which represents a team's score becomes score
        # date column name is changed to reflect its format 
        df.rename(columns={"w/l":"w_or_l", "tm":"score", "date":"yyyy_mm_dd"}, inplace=True)

        # removing special characters
        df.columns = df.columns.str.replace("%", "_pct")
        df.columns = df.columns.str.replace("/", "_per_") # notice that w/l was changed to w_or_l earlier so it wouldn't say w_per_l

        df.w_or_l = (df.w_or_l == "W").astype(int) # change data format to 1s and 0s

        # adjust data to reflect the game's pace, ignoring percentage based regular columns and all advanced columns since they consider pace intrinsically
        column_intervals = [["score", "fga"], ["3p", "3pa"], ["ft", "fta"], ["orb", "pf"]]
        for column_interval in column_intervals:
            df.loc[:, column_interval] = df.loc[:, column_interval].apply(lambda x: (x / df["pace"]) * 100)

        # restructure date data so that pandas understands it
        df["yyyy_mm_dd"] = pd.to_datetime(df["yyyy_mm_dd"])

        # add team_b2b column which equals 1 if a team is playing back to back games and 0 otherwise
        df.insert(df.columns.get_loc("score"), "team_b2b", (df["yyyy_mm_dd"].diff() == pd.Timedelta(days=1)).astype(int))

    return gamelogs

This is what the 2023 season looks like now for the Los Angeles Lakers:

In [10]:
gamelogs = preliminary_preprocessing(gamelogs)
gamelogs["LAL"]

,g,yyyy_mm_dd,team,location,opp,w_or_l,team_b2b,score,fg,fga,fg_pct,3p,3pa,3p_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,ortg,drtg,pace,ftr,3par,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_per_fga,drb_pct
0,1,2022-10-18,LAL,0,GSW,0,0,95.950704,40,82.746479,0.426,8.802817,35.211268,0.250,16.725352,22.007042,0.760,7.922535,48,23,12,4,21,15.845070,96.0,108.3,113.6,0.266,0.426,0.519,50.0,57.5,10.6,7.4,0.479,16.7,19.6,0.202,78.0
1,2,2022-10-20,LAL,1,LAC,0,0,93.269231,33,90.384615,0.351,8.653846,43.269231,0.200,21.153846,25.000000,0.846,7.692308,38,23,13,5,10,23.076923,93.3,99.1,104.0,0.277,0.479,0.460,39.6,69.7,12.5,9.8,0.399,8.7,14.3,0.234,75.0
2,3,2022-10-23,LAL,1,POR,0,0,103.792415,41,91.816367,0.446,5.988024,32.934132,0.182,15.968064,19.960080,0.800,9.980040,40,26,9,9,13,23.952096,103.8,105.8,100.2,0.217,0.359,0.516,43.5,63.4,9.0,15.8,0.478,11.4,20.4,0.174,69.8
3,4,2022-10-26,LAL,0,DEN,0,0,96.679688,40,87.890625,0.444,7.812500,29.296875,0.267,10.742188,15.625000,0.688,6.835938,47,25,8,4,14,15.625000,96.7,107.4,102.4,0.178,0.333,0.510,46.5,62.5,7.8,6.9,0.489,12.6,13.5,0.122,81.6
4,5,2022-10-28,LAL,0,MIN,0,0,96.682464,37,84.360190,0.416,8.530806,27.488152,0.310,18.009479,25.592417,0.704,11.374408,55,20,8,5,22,19.905213,96.7,105.2,105.5,0.303,0.326,0.506,52.9,54.1,7.6,8.5,0.466,17.9,25.5,0.213,75.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,2023-04-02,LAL,0,HOU,1,0,138.860104,52,96.373057,0.559,9.326425,31.088083,0.300,21.761658,29.015544,0.750,11.398964,51,36,4,8,5,9.326425,138.8,112.9,96.5,0.301,0.323,0.636,53.7,69.2,4.1,11.8,0.608,4.5,28.2,0.226,71.4
78,79,2023-04-04,LAL,0,UTA,1,0,138.036810,48,98.159509,0.500,12.269939,34.764826,0.353,27.607362,35.787321,0.771,11.247444,44,27,6,7,11,21.472393,125.0,123.1,97.8,0.365,0.354,0.606,51.8,56.3,5.6,12.3,0.563,9.0,23.9,0.281,84.6
79,80,2023-04-05,LAL,0,LAC,0,1,117.296223,45,88.469185,0.506,11.928429,28.827038,0.414,15.904573,23.856859,0.667,9.940358,44,27,4,3,14,16.898608,117.4,124.3,100.6,0.270,0.326,0.593,53.7,60.0,4.0,6.5,0.573,12.3,23.3,0.180,87.2
80,81,2023-04-07,LAL,1,PHO,1,0,117.475728,45,89.320388,0.489,17.475728,34.951456,0.500,12.621359,16.504854,0.765,9.708738,54,25,6,6,15,11.650485,117.4,103.8,103.0,0.185,0.391,0.608,53.5,55.6,5.8,8.8,0.587,13.1,22.7,0.141,77.2


Thirdly, we apply a rolling average to the stats so that each row's stats represent the average of the previous window_size games where window_size can be any integer. We also add a column that represents the team's win percent over the past window_size games, and another which indicates the team's current winning/losing streak. We then remove the first window_size rows since they don't have enough previous games to be set to the average of the last window_size games. Note: I tried letting all games stay in the dataset by finding the average of the last max(current game number, window_size) games, but this caused worse results.

In [11]:
def roll(gamelogs, window_size):
    for team_name, df in gamelogs.items():
        # set all applicable stats to the average of the previous window_size stats where window_size is an integer
        df.loc[:, "score":] = df.loc[:, "score":].shift().rolling(window=window_size).mean() # note: possible hyperparam could be a weighted mean
        
        # add new column team_lwsw_pct (last window_size win percentage) which equals the percentage of the last window_size games that were won
        df.insert(df.columns.get_loc("score"), "team_lwsw_pct", df.loc[:, "w_or_l"].shift().rolling(window=window_size).mean())

        # the following code finds the team's current win/loss streak using a clever trick from https://stackoverflow.com/questions/52976336/compute-winning-streak-with-pandas
        groups = df.w_or_l.ne(df.w_or_l.shift()).cumsum()
        df.insert(df.columns.get_loc("score"), "team_streak", None)
        df.team_streak = df.groupby(groups).cumcount() + 1
        df.team_streak = df.apply(lambda x: x.team_streak if x.w_or_l == 1 else -x.team_streak, axis=1)
        df.team_streak = df.team_streak.shift(fill_value=0)

        # deletes the first window_size rows since there were insufficient previous rows to set these rows to the average of the last window_size rows
        df.drop(df.index[:window_size], inplace=True)

    return gamelogs

The Lakers 2023 data now with a window_size of 5:

In [12]:
gamelogs = roll(gamelogs, 5)
gamelogs["LAL"]

,g,yyyy_mm_dd,team,location,opp,w_or_l,team_b2b,team_lwsw_pct,team_streak,score,fg,fga,fg_pct,3p,3pa,3p_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,ortg,drtg,pace,ftr,3par,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_per_fga,drb_pct
5,6,2022-10-30,LAL,1,DEN,1,0,0.0,-5,97.274900,38.2,87.439655,0.4166,7.957599,33.639931,0.2418,16.519786,21.636908,0.7596,8.761046,45.6,23.4,10.0,5.4,16.0,19.680861,97.30,105.16,105.14,0.2482,0.3846,0.5022,46.50,61.44,9.50,9.68,0.4622,13.46,18.66,0.1890,75.96
6,7,2022-11-02,LAL,1,NOP,1,0,0.2,1,102.930755,39.4,90.602885,0.4272,8.866440,32.757842,0.2784,16.460136,21.342276,0.7676,9.845943,46.2,23.6,8.6,5.6,13.4,20.002606,102.96,106.10,101.90,0.2366,0.3620,0.5138,46.80,60.38,8.40,9.98,0.4758,11.54,20.16,0.1820,75.26
7,8,2022-11-04,LAL,1,UTA,0,0,0.4,2,107.968913,41.8,91.874432,0.4488,9.110004,31.211597,0.2940,16.178034,20.883243,0.7724,10.479249,49.8,25.0,7.2,6.2,14.8,18.743589,105.74,107.20,101.36,0.2282,0.3396,0.5328,49.34,59.78,6.98,10.10,0.4980,12.52,21.62,0.1760,76.34
8,9,2022-11-06,LAL,1,CLE,0,0,0.4,-1,111.301811,41.6,90.748957,0.4560,10.196927,30.024563,0.3422,18.176529,22.706388,0.7910,10.352399,49.8,24.2,6.6,5.6,14.4,17.483803,109.08,113.04,100.58,0.2522,0.3304,0.5512,49.74,58.10,6.42,8.84,0.5120,12.30,21.82,0.2014,75.86
9,10,2022-11-07,LAL,0,UTA,0,1,0.4,-2,111.886193,41.6,90.501509,0.4592,10.228052,29.543674,0.3480,18.418530,23.764655,0.7676,10.977244,49.0,24.0,5.8,5.6,14.8,19.538086,109.66,114.28,100.18,0.2648,0.3258,0.5532,49.90,57.60,5.66,9.00,0.5154,12.64,23.20,0.2046,75.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,2023-04-02,LAL,0,HOU,1,0,0.8,2,120.169893,41.8,83.924243,0.5098,10.042517,26.478150,0.3888,25.017134,33.000093,0.7552,11.583764,45.0,24.2,8.2,3.6,14.6,16.925240,120.16,114.46,98.16,0.4040,0.3148,0.6122,53.12,58.22,8.32,6.40,0.5696,13.16,27.26,0.3066,77.34
78,79,2023-04-04,LAL,0,UTA,1,0,0.8,3,123.145166,44.4,88.971213,0.5102,10.281785,29.240482,0.3546,22.052392,29.453609,0.7486,12.847297,48.0,26.0,7.6,4.8,13.0,14.725485,123.14,114.48,97.78,0.3328,0.3308,0.6042,54.26,58.22,7.72,8.08,0.5684,11.54,29.78,0.2490,77.20
79,80,2023-04-05,LAL,0,LAC,0,1,0.8,4,126.585861,45.6,89.644781,0.5178,10.235773,29.110114,0.3546,23.407198,30.777739,0.7600,11.763452,46.2,27.4,7.0,5.4,13.0,16.103296,123.98,115.98,98.14,0.3442,0.3268,0.6132,53.80,59.96,6.96,9.12,0.5756,11.42,28.02,0.2612,79.02
80,81,2023-04-07,LAL,1,PHO,1,0,0.8,-1,127.163750,47.0,91.448788,0.5176,9.867222,27.248403,0.3652,22.562689,29.616908,0.7576,12.056609,47.4,28.0,6.8,5.2,12.2,15.669459,124.58,115.84,99.38,0.3236,0.2960,0.6082,55.04,59.32,6.70,9.00,0.5716,10.46,28.88,0.2466,80.68


Fourthly, to limit the number of features, I took each game, and set the stats equal to the team's stat value minus the opponent's. The alternative would have been storing the team's and the opponent's stats separately in the same row for each game, but this would have resulted in about twice as many columns.

In [13]:
def combine_team_with_opp(gamelogs):
    new_gamelogs = {}
    for team_name, df1 in gamelogs.items():
        # make new df to store the data the differential data between the team and opponent
        new_df = df1.copy() 

        # add columns for the opponenet stats where taking the differential isn't desired
        new_df.insert(new_df.columns.get_loc("score"), "opp_b2b", None)
        new_df.insert(new_df.columns.get_loc("score"), "opp_lwsw_pct", None)
        new_df.insert(new_df.columns.get_loc("score"), "opp_streak", None)

        for i, row in df1.iterrows(): # for each row in the team's table
            df2 = gamelogs[row.opp] # find the table with the opponent's data
            opp_stats = df2[df2.yyyy_mm_dd == row.yyyy_mm_dd] # find the row in the opponent's table corresponding to the current team row
            if not opp_stats.empty:
                # set the new_df stats to the differential between those of the team and opponent for the given game
                opp_stats_to_subtract = opp_stats.loc[:, "score":"drb_pct"]
                new_df.loc[i, "score":] = df1.loc[i, "score":] - opp_stats_to_subtract.values.flatten()

                # populate new_df's newly added opp_b2b, opp_lwsw_pct, and opp_streak columns
                opp_stats_to_transfer = opp_stats.loc[:,"team_b2b":"team_streak"]
                new_df.loc[i, "opp_b2b":"opp_streak"] = opp_stats_to_transfer.values.flatten()
        
        new_df.dropna(inplace=True) # delete rows where no opponent data was found

        new_df.opp_b2b = new_df.opp_b2b.astype(int)
        new_df.opp_streak = new_df.opp_streak.astype(int)

        new_gamelogs[team_name] = new_df

    return new_gamelogs

Here is the 2023 Lakers data now (with differentials):

In [14]:
gamelogs = combine_team_with_opp(gamelogs)
gamelogs["LAL"]

,g,yyyy_mm_dd,team,location,opp,w_or_l,team_b2b,team_lwsw_pct,team_streak,opp_b2b,opp_lwsw_pct,opp_streak,score,fg,fga,fg_pct,3p,3pa,3p_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,ortg,drtg,pace,ftr,3par,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_per_fga,drb_pct
5,6,2022-10-30,LAL,1,DEN,1,0,0.0,-5,0,0.8,2,-19.660694,-4.6,-0.208368,-0.0714,-6.572044,-1.038888,-0.1738,-0.527591,-3.216426,0.0698,-2.675940,-1.8,-5.2,3.4,1.8,2.2,-1.542581,-19.64,-9.72,4.70,-0.0394,-0.0130,-0.0922,-6.12,-5.20,3.02,3.02,-0.1094,1.34,-6.52,-0.0086,-5.08
6,7,2022-11-02,LAL,1,NOP,1,0,0.2,1,0,0.6,1,-18.379654,-4.2,-2.454204,-0.0646,-3.335167,1.510847,-0.1068,-1.452686,-0.745183,-0.0112,-1.301561,4.8,-3.4,0.0,2.0,-0.4,0.497880,-16.00,-8.48,6.10,-0.0056,0.0252,-0.0790,-2.86,-1.52,-0.42,3.60,-0.0820,-0.74,-6.20,-0.0150,3.76
7,8,2022-11-04,LAL,1,UTA,0,0,0.4,2,0,0.6,-1,-3.275054,1.4,3.237987,-0.0098,-5.018226,-9.717411,-0.0488,0.215299,-0.000858,-0.0038,0.021880,7.2,-2.2,-1.2,1.0,0.0,-5.679594,-5.50,-3.20,1.66,-0.0090,-0.1244,-0.0374,1.86,-7.68,-1.36,0.68,-0.0412,-0.66,-2.02,-0.0048,5.00
8,9,2022-11-06,LAL,1,CLE,0,0,0.4,-1,0,1.0,7,-8.864432,0.0,-0.890562,-0.0150,-4.420311,-7.660152,-0.0428,-1.452878,-2.727109,0.0086,0.010775,4.6,-1.8,1.6,0.2,0.4,-5.214943,-6.32,9.52,3.74,-0.0252,-0.0804,-0.0356,-2.60,-4.94,1.46,-1.78,-0.0392,-0.12,-2.04,-0.0126,-4.16
9,10,2022-11-07,LAL,0,UTA,0,1,0.4,-2,1,0.8,2,-6.856290,-1.0,1.376053,-0.0258,-5.914564,-11.108402,-0.0458,2.448784,5.197372,-0.0898,-0.254342,6.6,-2.6,-1.6,1.6,-0.8,-0.907505,-9.08,2.74,1.70,0.0472,-0.1366,-0.0580,-1.52,-4.68,-1.82,2.24,-0.0612,-1.48,-2.92,0.0172,-1.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,2023-04-02,LAL,0,HOU,1,0,0.8,2,0,0.2,1,4.791552,0.2,-6.769602,0.0322,1.723373,-1.708739,0.0952,4.435812,5.204822,0.0242,-2.830870,2.6,2.6,0.8,0.6,1.2,-3.234570,4.78,-17.24,2.12,0.0930,0.0038,0.0508,1.34,6.12,0.60,0.42,0.0462,1.12,-3.88,0.0754,1.30
78,79,2023-04-04,LAL,0,UTA,1,0,0.8,3,0,0.2,-2,10.726597,0.4,-1.021203,0.0260,0.990401,-1.745710,0.0510,5.997505,8.139285,-0.0222,2.324738,-0.2,0.0,4.0,-0.8,-1.8,-3.134813,10.74,-2.12,-3.18,0.0952,-0.0142,0.0370,1.04,-1.12,4.16,-2.28,0.0322,-1.28,4.26,0.0698,0.50
79,80,2023-04-05,LAL,0,LAC,0,1,0.8,4,0,0.4,-2,8.121446,3.4,6.456799,-0.0022,-5.755941,-6.191174,-0.0940,6.798789,8.689089,0.0178,4.777143,8.8,-2.4,-0.4,0.2,-2.8,-3.372794,5.50,-6.92,-0.42,0.0750,-0.1008,-0.0278,7.38,-10.86,-0.54,0.86,-0.0424,-3.36,10.72,0.0588,7.30
80,81,2023-04-07,LAL,1,PHO,1,0,0.8,-1,1,1.0,7,9.054066,5.6,1.691951,0.0362,-1.820204,-3.957740,-0.0002,2.027850,3.276004,-0.0208,0.678163,2.0,1.4,0.0,-1.6,0.2,-2.954106,6.46,7.84,2.94,0.0288,-0.0564,0.0224,4.70,-5.10,-0.32,-2.86,0.0238,-0.48,4.18,0.0164,7.20


Eventually we'll concatenate all the gamelogs for each team into one table, but this would cause there to be two copies of each game, one with the team's stat's minus those of the opponent, and vice versa. So, I've provided the following function which creates a unique game id (based on the date and the teams competing), and drops one of every pair of games sharing a unique_id.

In [15]:
def remove_duplicate_games(df):
    # the df contains two copies of each game and this unique_id helps us identify the duplicates
    df["unique_id"] = df.apply(lambda row: f"{row['yyyy_mm_dd']}_{('_'.join(sorted([row['team'], row['opp']])))}", axis=1)

    # randomize row order so that teams close to start of alphabet don't have more datapoints where their stats are the ones being subtracted from
    df = df.sample(frac=1) 

    df.drop_duplicates(subset="unique_id", keep="first", inplace=True)
    df.drop(columns="unique_id", inplace=True)
    df.sort_values(by="yyyy_mm_dd", inplace=True) # will prevent data leakage since test set will be the latter dates
    
    return df

This is the completed 2023 season dataset (with data for all teams):

In [16]:
df = pd.concat(gamelogs.values(), axis=0) # combine the data from all teams into one df
df = remove_duplicate_games(df)
df

,g,yyyy_mm_dd,team,location,opp,w_or_l,team_b2b,team_lwsw_pct,team_streak,opp_b2b,opp_lwsw_pct,opp_streak,score,fg,fga,fg_pct,3p,3pa,3p_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,ortg,drtg,pace,ftr,3par,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_per_fga,drb_pct
5,6,2022-10-28,HOU,0,POR,0,0,0.2,-1,0,0.8,-1,-9.412000,0.4,6.213922,-0.0484,0.161675,3.392222,-0.0308,-6.297069,-7.454230,0.0004,2.794000,3.0,0.6,-1.2,0.4,0.0,4.165788,-7.14,3.58,5.00,-0.1034,0.0114,-0.0620,-1.70,1.04,-1.34,0.64,-0.0528,-0.84,2.92,-0.0864,-3.26
5,6,2022-10-28,CHI,0,SAS,0,0,0.6,2,0,0.6,-1,-2.935662,-5.0,-4.864496,-0.0198,-2.722729,-5.682412,-0.0146,7.161116,6.083670,0.1068,-0.847065,1.6,-6.0,1.4,1.2,-1.6,1.156209,-2.96,-7.38,-3.56,0.0692,-0.0448,-0.0048,-0.04,-6.72,1.72,3.20,-0.0332,-0.72,-3.08,0.0820,3.38
5,6,2022-10-28,DEN,1,UTA,1,0,0.6,1,0,0.8,1,-5.352237,-0.2,-8.057385,0.0364,-2.523056,-8.927306,0.0100,-1.849923,-3.667909,0.0572,-4.876950,-5.8,0.0,-2.4,-2.6,-1.0,-1.565532,-0.54,10.64,0.72,-0.0196,-0.0644,0.0284,-1.46,0.44,-2.02,-3.12,0.0292,0.08,-8.78,-0.0046,9.04
5,6,2022-10-28,TOR,1,PHI,0,0,0.6,2,0,0.2,-1,-0.616036,0.6,3.822477,-0.0160,0.973870,-0.648810,0.0512,-2.456043,-1.026521,-0.0616,2.227756,4.4,1.8,2.8,0.4,-0.4,5.444301,-0.64,-6.86,0.62,-0.0246,-0.0270,-0.0224,3.62,3.44,2.90,0.84,-0.0146,-0.72,4.38,-0.0386,4.32
5,6,2022-10-29,MEM,0,UTA,0,0,0.8,2,1,0.6,-1,6.541409,1.0,-1.507555,0.0236,1.773440,-1.946321,0.0658,1.940670,2.459695,0.0160,-0.860956,-2.8,-4.4,-4.0,-1.2,-2.2,-4.068103,9.08,8.50,-0.92,0.0300,-0.0160,0.0350,1.42,-11.10,-3.60,-1.70,0.0338,-1.38,-0.18,0.0252,4.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,82,2023-04-09,BRK,1,PHI,0,0,0.8,2,0,0.6,1,-1.862465,-0.8,3.390219,-0.0374,1.253196,5.510577,-0.0322,-0.163310,1.961990,-0.0810,0.370871,2.0,3.6,2.2,0.4,-4.6,-3.253325,0.50,-15.06,1.82,0.0140,0.0414,-0.0360,-2.10,10.14,2.50,-0.04,-0.0338,-4.28,-0.90,-0.0092,-2.24
81,82,2023-04-09,DET,0,CHI,0,0,0.2,1,0,0.6,1,-0.412519,-4.4,-4.729632,-0.0196,1.360216,-0.064197,0.0480,6.185382,7.870945,0.0116,3.152473,-1.8,-0.8,-2.4,-0.8,5.2,10.752256,-0.44,15.62,-1.18,0.1088,0.0194,0.0072,2.10,4.14,-2.50,-2.60,-0.0086,4.64,8.46,0.0864,-5.24
81,82,2023-04-09,LAL,1,UTA,1,0,0.8,1,1,0.2,1,13.629278,3.4,1.736318,0.0358,2.505589,-1.218176,0.0972,2.708823,4.365961,-0.0394,0.885195,-0.6,2.6,0.8,2.8,-3.0,-6.352064,13.62,-0.18,-1.76,0.0422,-0.0212,0.0462,0.78,0.98,0.84,3.54,0.0492,-2.54,2.98,0.0256,-0.64
81,82,2023-04-09,TOR,1,MIL,1,0,0.4,-2,0,0.6,-1,1.016732,0.4,4.718366,-0.0038,-7.497825,-10.048019,-0.1060,3.627422,2.741549,0.0870,1.900344,-5.8,-1.4,3.8,1.2,-5.0,2.302472,1.00,-5.96,-4.52,0.0188,-0.1224,-0.0314,-4.26,-4.44,4.26,3.80,-0.0470,-4.06,2.12,0.0312,-6.28


This function combines all of the data processing steps we've discussed above to get the dataset for an entire season.

In [17]:
def get_df_for_season(year, window_size):
    gamelogs = combine_and_extract_relevant_data(year)
    gamelogs = preliminary_preprocessing(gamelogs)
    gamelogs = roll(gamelogs, window_size)
    gamelogs = combine_team_with_opp(gamelogs)
    df = pd.concat(gamelogs.values(), axis=0) 
    df = remove_duplicate_games(df)
    
    return df

To get the final dataset, we use get_df_for_season to calculate and combine the dfs for as many seasons as desired. This function finds the saved data for all the years that we've scraped, and combines them.

In [18]:
def get_combined_df(window_size):
    years = sorted([folder for folder in os.listdir("gamelogs") if folder.isdigit()]) # ignores non-numerical folder names (only wants years)
    dfs = [get_df_for_season(year, window_size) for year in years]
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

Here's the final dataset:

In [19]:
combined_df = get_combined_df(35) # eventually I'll tune the window_size hyperparam properly but for now 35 seems to give decent results
combined_df

,g,yyyy_mm_dd,team,location,opp,w_or_l,team_b2b,team_lwsw_pct,team_streak,opp_b2b,opp_lwsw_pct,opp_streak,score,fg,fga,fg_pct,3p,3pa,3p_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,ortg,drtg,pace,ftr,3par,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_per_fga,drb_pct
0,36,2015-01-06,MIL,1,PHO,0,0,0.514286,1,0,0.542857,2,-4.973584,-1.914286,-1.860979,0.001400,-2.980634,-7.345455,-0.004400,-0.435346,0.210657,-0.028914,-0.024342,-1.085714,1.942857,0.514286,-0.371429,1.571429,0.486528,-5.217143,-3.568571,-2.782857,0.011657,-0.077029,-0.014829,0.711429,8.197143,0.814286,-0.188571,-0.014057,1.794286,0.434286,0.002229,0.185714
1,36,2015-01-07,LAC,1,LAL,1,0,0.657143,-1,0,0.314286,-1,5.758739,0.571429,-4.136076,0.033600,3.437529,7.556796,0.028086,0.076784,0.048295,0.010200,-2.999231,-1.428571,3.942857,0.171429,0.114286,0.057143,-0.887030,6.365714,-6.594286,-1.014286,0.018714,0.098086,0.050400,1.334286,9.568571,0.285714,0.582857,0.055000,0.548571,-3.797143,0.015029,3.851429
2,36,2015-01-07,DEN,1,ORL,1,0,0.428571,2,0,0.371429,-3,4.668076,0.971429,3.094143,-0.019400,0.214161,4.095262,-0.050257,5.218960,6.618528,0.013257,3.909594,5.400000,0.971429,-0.171429,1.228571,0.371429,2.270187,4.071429,0.331429,3.425714,0.065000,0.035914,-0.007971,2.505714,1.145714,-0.477143,1.708571,-0.020057,-0.794286,7.208571,0.052257,-1.408571
3,36,2015-01-07,UTA,0,CHI,1,0,0.342857,-1,0,0.714286,3,-3.325002,-1.142857,-0.332655,0.002829,-0.336315,0.752679,-0.027686,-3.383529,-3.130484,-0.023286,0.110759,-3.828571,-1.828571,0.142857,-1.314286,0.142857,1.111393,-2.031429,6.617143,-3.308571,-0.036314,0.008371,-0.007543,0.200000,-2.957143,0.422857,-0.434286,0.001057,0.762857,0.488571,-0.039600,1.468571
4,36,2015-01-08,HOU,0,NYK,1,1,0.685714,1,1,0.085714,-13,3.839853,-0.085714,-2.704807,-0.009086,4.335281,12.971833,-0.020714,3.228007,6.795625,-0.073171,1.516457,4.600000,-0.514286,1.971429,0.771429,2.628571,0.218496,3.477143,-9.951429,4.182857,0.086743,0.155657,0.016943,3.397143,-1.108571,1.717143,0.794286,0.016943,1.568571,3.814286,0.043714,2.385714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5927,82,2023-04-09,POR,1,GSW,0,1,0.342857,-3,0,0.571429,2,-3.624248,-2.571429,-2.331081,-0.011057,-2.956363,-4.946834,-0.029743,3.422736,3.526018,0.026086,-2.884938,-7.114286,-5.942857,-0.085714,1.114286,-2.714286,-1.432306,-2.982857,8.120000,-1.245714,0.048000,-0.043886,-0.015200,-5.934286,-10.134286,0.031429,2.080000,-0.025657,-1.957143,-6.674286,0.045429,-2.374286
5928,82,2023-04-09,IND,0,NYK,1,0,0.314286,-3,0,0.628571,-1,-5.996579,0.028571,-1.549230,-0.012486,-1.650229,-3.597082,-0.013714,-0.820082,-2.208076,0.038114,-1.381215,-4.485714,3.514286,1.142857,1.571429,1.685714,0.336112,-5.022857,4.640000,4.097143,-0.016343,-0.035314,-0.016714,-5.685714,7.834286,0.897143,1.985714,-0.021343,1.165714,-2.617143,-0.002514,-7.688571
5929,82,2023-04-09,PHO,1,LAC,0,0,0.657143,-1,1,0.571429,2,-5.150398,-0.114286,3.025107,-0.022371,-2.101557,-2.307004,-0.036629,-1.996142,-1.982844,0.001057,2.488661,3.485714,2.314286,-0.057143,1.628571,-0.371429,2.030569,-4.000000,-6.277143,0.902857,-0.029714,-0.037229,-0.038029,1.877143,5.620000,-0.054286,3.451429,-0.036371,-0.545714,4.128571,-0.028657,0.885714
5930,82,2023-04-09,HOU,0,WAS,1,0,0.314286,2,0,0.428571,1,-2.931410,-0.742857,2.580429,-0.026714,-2.379980,-3.022438,-0.043000,1.607413,3.554552,-0.059857,4.645048,3.000000,-3.085714,0.485714,-0.600000,1.342857,2.494564,-2.937143,3.782857,0.622857,0.031657,-0.043829,-0.039229,2.465714,-6.377143,0.468571,0.088571,-0.042114,0.471429,8.317143,0.012143,0.482857


Here we remove irrelevant features (which were useful for organization/preprocessing but are no longer wanted), and split the dataset into training and testing sets (notice that shuffle is set to false to prevent data leakage since the data is temporally dependent).

In [20]:
df = combined_df.copy() # make copy so the model can be run again (with different features/test_size) without waiting for combined_df to be computed again
df = df.drop(columns=["g", "yyyy_mm_dd", "team", "opp", "pace"]) # remove irrelevant columns
X = df.drop(columns="w_or_l")
y = df["w_or_l"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

Below, I use logistic regression to predict the results of these games. I'm using time series cross validation to provide a realistic estimation of the model's potential performance on unseen data (this means games that happened in the future are not used to predict past games as could be the case with standard cross validation).

In [21]:
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits) 
model = LogisticRegression(max_iter=500) # need about 500 iterations so it doesn't fail to converge

total_accuracy = 0
for i, [train_index, validate_index] in enumerate(tscv.split(X_train)):
    X_train_fold, X_validate = X_train.iloc[train_index], X_train.iloc[validate_index]
    y_train_fold, y_validate = y_train.iloc[train_index], y_train.iloc[validate_index]

    model.fit(X_train_fold, y_train_fold)
    y_pred = model.predict(X_validate)

    accuracy = accuracy_score(y_validate, y_pred)
    print(f"fold {i} accuracy: {accuracy}")
    total_accuracy += accuracy

print(f"average accuracy: {total_accuracy / n_splits}")

fold 0 accuracy: 0.6747638326585695
fold 1 accuracy: 0.6356275303643725
fold 2 accuracy: 0.6882591093117408
fold 3 accuracy: 0.650472334682861
fold 4 accuracy: 0.6369770580296896
average accuracy: 0.6572199730094468
